In [ ]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn import model_selection, metrics 
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import csv
from scipy.stats import spearmanr
import numerapi

napi = numerapi.NumerAPI()

# train_csv_path = "numerai_training_data_int8.csv"

# napi.download_dataset("numerai_training_data_int8.csv", train_csv_path)


with open('numerai_training_data_int8.csv', 'r') as f:
    column_names = next(csv.reader(f))
    dtypes = {x: np.float32 for x in column_names if
              x.startswith(('feature', 'target'))}
data = pd.read_csv('numerai_training_data_int8.csv', dtype=dtypes, header=0, index_col=0)

data_r = data.sample(n=50000)


features = [f for f in data_r.columns if f.startswith("feature")]
target = "target_nomi_20"
# data["erano"] = data.era.str.slice(3).astype(int)
# eras = data.erano
data_r["erano"] = data_r.era.astype(int)
eras = data_r.erano


class TimeSeriesSplitGroups(_BaseKFold):
    def __init__(self, n_splits=5):
        super().__init__(n_splits, shuffle=False, random_state=None)

    def split(self, X, y=None, groups=None):
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_list = np.unique(groups)
        n_groups = len(group_list)
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds ={0} greater"
                 " than the number of samples: {1}.").format(n_folds,
                                                             n_groups))
        indices = np.arange(n_samples)
        test_size = (n_groups // n_folds)
        test_starts = range(test_size + n_groups % n_folds,
                            n_groups, test_size)
        test_starts = list(test_starts)[::-1]
        for test_start in test_starts:
            
            yield (indices[groups.isin(group_list[:test_start])],
                   indices[groups.isin(group_list[test_start:test_start + test_size])])


def spearman(y_true, y_pred): 
    return spearmanr(y_pred, y_true).correlation 


#duurt lang

cv_score = []
models = []
for lr in [0.006, 0.008, 0.01, 0.012, 0.014]:
    for cs in [0.06, 0.08, 0.1, 0.12, 0.14]:
        for md in [4, 5, 6]:
            models.append(XGBRegressor(colsample_bytree=cs, learning_rate=lr, n_estimators=2000, max_depth=md, nthread=8))



for model in models:
    score = np.mean(model_selection.cross_val_score(
                model,
                data_r[features],
                data_r[target],
                cv=TimeSeriesSplitGroups(5),
                n_jobs=1,
                groups=eras,
                scoring=metrics.make_scorer(spearman, greater_is_better=True)))
    cv_score.append(score)
    print(cv_score)

/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index i

[0.03669356006935219]


/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning:

[0.03669356006935219, 0.03583299760499424]


/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning:

...

/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning:

[0.03669356006935219, 0.03583299760499424, 0.034357369553894415, 0.03681434114207996, 0.03792851348276108, 0.033672838134681735, 0.03819564017655254, 0.03677002621850554, 0.03660025532705576, 0.03511901797340759, 0.035470068494468716, 0.03235197926662401, 0.03517107368471002, 0.034984126683089416, 0.031375467312485444, 0.03470190078320514, 0.03261276791646969, 0.03337639018262282, 0.035177376485485995, 0.03661165998738767, 0.032700891778454026, 0.03476912928319599, 0.03623900486295814, 0.03295127951686498, 0.035667223772952825, 0.03533558045314393, 0.030455492434456017, 0.03342493430976513, 0.03271925970712878, 0.033381343424535435, 0.034433190894000994, 0.031573567517534026, 0.030876202794823282, 0.03479032671372004, 0.034882795435285996, 0.030537288419470666, 0.037163393903296135, 0.03301150472150275, 0.03198830624844857, 0.03517175070394727, 0.032394859618823775, 0.028473192145941913, 0.034132470078724386, 0.02946887458021641, 0.02812324073397346, 0.0338414190017958, 0.0304777540823

/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning:

[0.03669356006935219, 0.03583299760499424, 0.034357369553894415, 0.03681434114207996, 0.03792851348276108, 0.033672838134681735, 0.03819564017655254, 0.03677002621850554, 0.03660025532705576, 0.03511901797340759, 0.035470068494468716, 0.03235197926662401, 0.03517107368471002, 0.034984126683089416, 0.031375467312485444, 0.03470190078320514, 0.03261276791646969, 0.03337639018262282, 0.035177376485485995, 0.03661165998738767, 0.032700891778454026, 0.03476912928319599, 0.03623900486295814, 0.03295127951686498, 0.035667223772952825, 0.03533558045314393, 0.030455492434456017, 0.03342493430976513, 0.03271925970712878, 0.033381343424535435, 0.034433190894000994, 0.031573567517534026, 0.030876202794823282, 0.03479032671372004, 0.034882795435285996, 0.030537288419470666, 0.037163393903296135, 0.03301150472150275, 0.03198830624844857, 0.03517175070394727, 0.032394859618823775, 0.028473192145941913, 0.034132470078724386, 0.02946887458021641, 0.02812324073397346, 0.0338414190017958, 0.0304777540823

/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/home/mathiasvanaelten/miniconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
